In [1]:
import csv
import pandas as pd
import numpy as np
import re
import copy
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

2024-07-03 23:09:40,936 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-07-03 23:09:40,937 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-03 23:09:40,963 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 3b674e250eaff3ed305bb955cabf2c95 and a total number of 980 components indexed
/root/miniconda3/envs/qwen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TDOO 原版用14B-Chat，太大加载不了，只能换Int4版本
model_dir = '/mnt/workspace/Tongyi-Finance-14B-Chat-Int4'

In [3]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

In [4]:
new_question_file_dir = '/mnt/workspace/intermediate/A01_question_classify.csv'
new_question_file = pd.read_csv(new_question_file_dir,delimiter = ",",header = 0)

In [5]:
company_file_dir = '/mnt/workspace/files/AF0_pdf_to_company.csv'
company_file = pd.read_csv(company_file_dir,delimiter = ",",header = 0)

In [6]:
company_data_csv_list = list()
company_index_list = list()
company_name_list = list()
for cyc in range(len(company_file)):
    company_name_list.append(company_file[cyc:cyc+1]['公司名称'][cyc])
    company_data_csv_list.append(company_file[cyc:cyc+1]['csv文件名'][cyc])
    temp_index_cp = tokenizer(company_file[cyc:cyc+1]['公司名称'][cyc])
    temp_index_cp = temp_index_cp['input_ids']
    company_index_list.append(temp_index_cp)

In [7]:
g = open('/mnt/workspace/intermediate/A02_question_classify_entity.csv', 'w', newline='', encoding = 'utf-8-sig') 
csvwriter = csv.writer(g)
csvwriter.writerow(['问题id','问题','分类','对应实体','csv文件名'])

24

In [8]:
for cyc in range(len(new_question_file)):
    
    tempw_id = new_question_file[cyc:cyc+1]['问题id'][cyc]
    tempw_q = new_question_file[cyc:cyc+1]['问题'][cyc]
    tempw_q_class = new_question_file[cyc:cyc+1]['分类'][cyc]
    tempw_entity = 'N_A'
    tempw_csv_name = 'N_A'
    
    
    if new_question_file[cyc:cyc+1]['分类'][cyc] == 'Text':
        temp_index_q = tokenizer(new_question_file[cyc:cyc+1]['问题'][cyc])
        temp_index_q = temp_index_q['input_ids']
        q_cp_similarity_list = list()
        for cyc2 in range(len(company_file)):
            temp_index_cp = company_index_list[cyc2]
            temp_simi = len(set(temp_index_cp) &set(temp_index_q))/ (len(set(temp_index_cp))+len(set(temp_index_q)))
            q_cp_similarity_list.append(temp_simi)
            
            
        t = copy.deepcopy(q_cp_similarity_list) 
        max_number = []
        max_index = []
        
        for _ in range(1):
            number = max(t)
            index = t.index(number)
            t[index] = 0
            max_number.append(number)
            max_index.append(index)
        t = []
        tempw_entity = company_name_list[max_index[0]]
        tempw_csv_name = company_data_csv_list[max_index[0]]
       
        csvwriter.writerow([str(tempw_id),str(tempw_q),tempw_q_class,tempw_entity,tempw_csv_name])
    elif new_question_file[cyc:cyc+1]['分类'][cyc] == 'SQL':
        csvwriter.writerow([str(tempw_id),str(tempw_q),tempw_q_class,tempw_entity,tempw_csv_name])
    else:
        find_its_name_flag = 0
        for cyc_name in range(len(company_name_list)):
            if company_name_list[cyc_name] in tempw_q:
                tempw_entity = company_name_list[cyc_name]
                tempw_csv_name = company_data_csv_list[cyc_name]
                csvwriter.writerow([str(tempw_id),str(tempw_q),tempw_q_class,tempw_entity,tempw_csv_name])
                find_its_name_flag = 1
                break
        if find_its_name_flag == 0:
            csvwriter.writerow([str(tempw_id),str(tempw_q),tempw_q_class,tempw_entity,tempw_csv_name])


In [9]:
g.close()
print('A02_finished')

A02_finished
